### From the insight analisys, we came up with the following Dashboard structure:

#### Resource distribution per institution
- Barchart with total value and iCSHG value per institution sorted by total value
- Highlights nn projects, xxMM of total value, pp% iCSHG total yMM
- Interactivity: total and iCSHG value per project, sorting per iCSHG value as well

#### Drew down to resource distribution view
- Distribution by activity field and project mean iCSHG value per field: highlights professional education
- Table with aggregated finances with filter for top 5: hightligths revenue outliers
- optional: area chart

#### Institution assistence characteristics
- xx k benefited, R$ yk per benefited, hh
- Scatterplot relating per_capita x carga horaria cutoff and segmentation of benefited quantity
- (cutoff + per_capita x benefited tradeoff)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = 200
from sklearn.decomposition import PCA

import plotly.offline as pyoff
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

In [ ]:
compfin = pd.read_pickle('compfin.pkl')

### Resource distribution per institution

In [ ]:
# restrinjo a analise aos projetos com dados
compfin = compfin[compfin.has_detailed_info]
compfin = compfin.assign(id_org = lambda x: x.organizacao + ": " + x.nome_projeto)
compfin=compfin.sort_values('valor_total', ascending = True)

In [ ]:
compfin.id_org = compfin.id_org.str.strip() + " "

In [ ]:
compfin.set_index('id_org', inplace=True)

In [ ]:
compfin[compfin.organizacao=="Verdescola"]

In [ ]:
help(go.Layout)

In [ ]:
trace1 = go.Bar(
                x=compfin.valor_total,
                y=compfin.organizacao.str.strip() + " ",
                orientation = 'h',
                text = compfin.nome_projeto.str.strip(),
                name = "Valor Total"
                )

trace2 = go.Bar(
                x=compfin.solicitado_iCSHG,
                y=compfin.organizacao.str.strip() + " ",
                orientation = 'h',
                text = compfin.nome_projeto.str.strip(),
                name = "Solicitado iCSHG"
                )

data = [trace1, trace2]

pyoff.init_notebook_mode(connected=True)

#pyoff.iplot({
#    "data": data,
#    "layout": go.Layout(
#                        title="Total de 29 projetos, 37 MM e 22% de Contribuição CSHG", 
#                        barmode = "overlay",
#                        barnorm = "percent",
#                        margin=dict(
#                                    l=220,
#                                    r=10,
#                                    t=140,
#                                    b=80
#                                    )
#                        )
#})

In [ ]:
pyoff.plot({
    "data": data,
    "layout": go.Layout(
                        title="Total de nn projetos, xx MM e pp% de Contribuição CSHG", 
                        barmode = "overlay",
                        barnorm = "percent",
                        margin=dict(
                                    l=220,
                                    r=10,
                                    t=140,
                                    b=80
                                    )
                        )
}, auto_open=True)

### Drew down to resource distribution view

In [ ]:
consolidated1 = compfin[compfin.valor_total > 0].groupby(['area']).\
nome_projeto.count().to_frame()
consolidated1.rename(columns={"nome_projeto": "counts"}, inplace=True)

consolidated2 = compfin[compfin.valor_total > 0].groupby(['area']).\
solicitado_iCSHG.mean().to_frame()
consolidated2.rename(columns={"solicitado_iCSHG": "means"}, inplace=True)


consolidated3 = compfin[compfin.valor_total > 0].groupby(['area']).\
solicitado_iCSHG.sum().to_frame()
consolidated3.rename(columns={"solicitado_iCSHG": "sums"}, inplace=True)


consolidated = pd.concat([consolidated1,consolidated2,consolidated3], axis=1)
consolidated.reset_index(inplace=True)
consolidated.sort_values('counts', inplace=True)

In [ ]:
consolidated.head()

In [ ]:
# fazer esse aqui: https://plot.ly/python/horizontal-bar-charts/

trace1 = go.Bar(
                y=consolidated.counts,
                x=consolidated.area.str.strip(),
                )

trace2 = go.Scatter(
                y=consolidated.means,
                x=consolidated.area.str.strip(),
                text = "total CSHG: "+consolidated.sums.astype('str'),
                name = "Solicitado iCSHG",
                yaxis='y2'
                )

data = [trace1, trace2]

pyoff.init_notebook_mode(connected=True)

pyoff.plot({
    "data": data,
    "layout": go.Layout(title="Quantidade de projetos por área",
                       yaxis=dict(
        title='yaxis title'
    ),
    yaxis2=dict(
                title='yaxis2 title',
                titlefont=dict(
                color='rgb(148, 103, 189)'),
                overlaying = 'y',
                side = 'right'
            )
    )
        
}, auto_open=True)

- ou dá um colorido ou muda para linha

In [ ]:
consolidated.sums.astype('str')

In [ ]:
trace1 = {"x": compfin.receitas_2015, 
          "y": compfin.organizacao.str.strip()+" ", 
          "marker": {"color": "blue", "size": 12}, 
          "mode": "markers", 
          "name": "Receitas 2015", 
          "type": "scatter"
         }

trace2 = {"x": compfin.despesas_2015, 
          "y": compfin.organizacao.str.strip()+" ", 
          "marker": {"color": "red", "size": 12}, 
          "mode": "markers", 
          "name": "Despesas 2015", 
          "type": "scatter"
         }

trace3 = {"x": compfin.ativo_2015, 
          "y": compfin.organizacao.str.strip()+" ", 
          "marker": {"color": "green", "size": 12}, 
          "mode": "markers", 
          "name": "Ativo 2015", 
          "type": "scatter"
         }

trace4 = {"x": compfin.pl_2015, 
          "y": compfin.organizacao.str.strip()+" ", 
          "marker": {"color": "black", "size": 12}, 
          "mode": "markers", 
          "name": "PL 2015", 
          "type": "scatter"
         }

data = [trace1, trace2, trace3, trace4]
layout = {"title": "Finanças", 
          "xaxis": {"title": "(Reais)", "type": "log"}, 
          "yaxis": {"title": "Instituição"},
          "margin": dict(
                                    l=220,
                                    r=10,
                                    t=140,
                                    b=80
                                    )
         }

fig = go.Figure(data=data, layout=layout)

pyoff.plot({
    "data": data,
    "layout": layout
}, auto_open=True)

### Institution assistence characteristics

In [ ]:
hover_text = []
bubble_size = []

for index, row in compfin.iterrows():
    hover_text.append(('Organização: {organizacao} <br>'+
                      'Projeto: {nome_projeto} <br>'+
                      'Valor total: {valor_total} <br>').format(organizacao = row['organizacao'],
                               nome_projeto = row['nome_projeto'],
                               valor_total = row['valor_total']))
    bubble_size.append(row['beneficiarios'])

compfin['text'] = hover_text
compfin['size'] = bubble_size
sizeref = 1

data = []
for area in set(compfin.area):
    data.append(
        go.Scatter(
        x=compfin['carga_horaria'][compfin['area'] == area],
        y=compfin['per_capita'][compfin['area'] == area],
        mode='markers',
        name=area,
        text=compfin['text'][compfin['area'] == area],
        marker=dict(
            symbol='circle',
            sizemode='area',
            sizeref=1,
            size=compfin['size'][compfin['area'] == area],
            line=dict(
                width=2
                ),
                    )
                )
    )

layout = go.Layout(
    title='Carga horária, valor per capita e beneficiários',
    xaxis=dict(
        title='Carga horária (h)',
        gridcolor='rgb(255, 255, 255)',
        #range=[2.003297660701705, 5.191505530708712],
        #type='log',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Per capita (R$)',
        gridcolor='rgb(255, 255, 255)',
        #range=[36.12621671352166, 91.72921793264332],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)

pyoff.plot({
    "data": data,
    "layout": layout
}, auto_open=True)

In [ ]:
#hora_per_capita = compfin.carga_horaria/compfin.per_capita
#hora_per_capita = compfin[compfin.index.isin(hora_per_capita.nlargest(5).index)]
#x_= hora_per_capita.carga_horaria 
#y_ = hora_per_capita.per_capita
#annotations = []
#for x_,y_ in zip(x_.values,y_.values):
#    annotations.append(
#            dict(
#            x=x_,
#            y=y_,
#            xref='x',
#            yref='y',
#            text='hr/per_capita',
#            showarrow=True,
#            arrowhead=7,
#            ax=0,
#            ay=-40
#            )
#    )